In [1]:
# !pip install pip --upgrade
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import pandas as pd
# !pip install pyopenssl --upgrade
# !pip install --upgrade smart_open
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# for processing
import re
import nltk
from nltk.tokenize import word_tokenize


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

In [2]:
%%time
embeddings_dict_6B_50D = {}
with open("glove.6B/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = ' '.join(values[:-50]).lower().strip()
        vector = np.asarray(values[-50:], "float32")
        embeddings_dict_6B_50D[word] = vector

Wall time: 3.84 s


In [3]:
# # find the most similar words to a given word
# def find_closest_words(embedding_vec, embeddings_dict):
#     # This closest word is always the same word
#     return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding_vec))

In [4]:
# find_closest_words(embeddings_dict_6B_50D["twig"] - embeddings_dict_6B_50D["branch"] + embeddings_dict_6B_50D["hand"], embeddings_dict_6B_50D)[:5]

In [5]:
cfpb = pd.read_csv("complaints-2023-07-04_01_06.csv")
cfpb = cfpb[~cfpb['Consumer complaint narrative'].isna()]
cfpb

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
4,05/31/23,Debt collection,Auto debt,Written notification about debt,Didn't receive enough information to verify debt,"U.S.C in Violations 1. Title 15 USC 689n ( a ) ( b ) Unlawful acts and Omissions, breach of fiduciary duty 2. Title 12 Ch.X Part 1006.34 ( a ) ( c ), c ( 2 ) ( v ), c ( 3 ) ( ii ) Validation Information Required 3. Title 18 USC 1001 a ( 1 ) Statements or entries in generally 4. Title 15 USC 1681s-2 ( a ) ( b ), ( f ) ( 2 ) ( b ) ( f ) ( 3 ) Responsibilities of Furnishers of information to consumer reporting agencies 5. Title 15 USC 1611 ( 1 ) Criminal liability for willful and knowing violation 6. Title 18 USC 242 Deprivation of rights under color of law Denied Rights to Discovery 7.Title 15 USC 1691 ( a ), ( a ) ( 3 ) Scope of Prohibition 8. Title 15 USC 1681o, ( a ), ( a ) ( 1 ) Civil Liability for negligent noncompliance Requested verification Information. \n1. They did not produce a promissory note ; there was no verification. Noncompliance 2. Did they produce a contract signed by XXXX XXXX XXXX and by XXXX ; No verification or disclosure was provided. Noncompliance 3. Did they produce any verification of their claim ; affidavit/signed invoice was not provided willfully. \n\n4. When asked who the original creditor was, did they state clearly whom it was and their address ; they willfully denied full disclosure and failed to give proper verification information. Noncompliance 5. There was no Unreserved acceptance by both parties. ( I reserved my rights to rescind ) 6. Was equal consideration exchanged? No disclosure or verification was given. \n\n\n7. Have any other accounts been opened in my name at my address with or without my consent? No disclosure or verification was given. \n\n8. They disclosed my personal information to consumer reporting agencies. They reported under three different names : one name per agency. Reporting unverified incorrect information causing damage. \n\n\n9. Has the account been securitized? What is the Cusip-number? No disclosure or verification given. Noncompliance",Company has responded to the consumer and the CFPB and chooses not to provide a public response,Bridgecrest Acceptance Corporation,AL,361XX,None,Consent provided,Web,05/31/23,Closed with explanation,Yes,NaN,7045946
6,05/03/23,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Information belongs to someone else,"On XX/XX/XXXX, I submitted a request to block items on my consumer report as I am a victim of identity theft. This was not a dispute, however, the FCRA has chosen to forgo XXXX regulation and apply their company policy to treat this as a dispute. In fact, when I called them, they stated that their policy supersedes XXXX law ( which states ) ( a ) Block A consumer reporting agency should block the reporting of any information in the file of a consumer that the consumer identifies as information that resulted from alleged identity theft, not later than 4 business days after the date receipt by such agency of- ( 1 ) appropriate proof of the identity of the consumer ; ( 2 ) a copy of an identity theft report, ( 3 ) the identification of such information by the consumer, and ( 4 ) a statement by the consumer that the information is not information relating to any transaction by the consumer. I submitted proof that my social security number, fingerprints, birthdate, addresses, and other identifying information were affected by the XXXX XXXX data breach as well as XXXX 's XXXX data breach. I submitted all the required documents per 15usc1681c and yet the CRA has failed to block this information and it has been over 4 days. Again, this is NOT A DISPUTE.",Company has re

##### We dont remove stop words,  perform lemmatization or stemmingfor GloVe
Context: GloVe vectors capture the context of words by considering their co-occurrences with other words. Removing stopwords might lead to loss of some context, especially for tasks where the semantics of the full sentence are important. For instance, in sentiment analysis, removing stopwords could potentially alter the sentiment of a sentence.

Semantic Meaning: Some stopwords carry semantic meaning. For instance, the word "not" is generally considered a stopword, but it is also a negation word that can completely reverse the meaning of a sentence. If you remove it, you might change the meaning of your data.

Word Frequency: GloVe vectors are trained on a corpus where the stopwords are usually not removed. As a result, the embeddings for these words can capture useful semantic and syntactic relationships because of their high frequency of occurrence.

Dimensionality: On the other hand, removing stopwords can help in reducing the dimensionality of your data and might make your model faster and more efficient, especially if you have a very large corpus.


Lemmatization and stemming are techniques used to reduce words to their root form. These techniques can be useful for certain tasks where you want to treat different inflections of a word (like "run", "running", "runs") as the same word. However, when using pre-trained word embeddings like GloVe, you should be aware that the embeddings for the original words and their stemmed/lemmatized versions are not the same, and in some cases, the stemmed/lemmatized version might not even be in the vocabulary.

Additionally, lemmatization and stemming can sometimes change the meaning of the word or make it lose its context. For instance, the word "meeting" can be a verb (as in "I am meeting my friends") or a noun (as in "I have a meeting"). If you stem it to "meet", it might lose this contextual information.

In [6]:
from nltk.tokenize import RegexpTokenizer

def text_normalizer(text):
    if text:
        # Use NLTK RegexpTokenizer for tokenization. 
        # This tokenizer splits the text by white space and also keeps tokens like "wasn't" and "don't".
        tokenizer = RegexpTokenizer(r'\b\w[\w\'-]*\w\b|\w')
        words = tokenizer.tokenize(text)

        # Clean up any token with repeating characters like '666', 'aaa', '!!!!!!', substitute them with empty string ''.
        # This includes 'XXXX' maskings in the text created by CFPB.
        words = [re.sub(r'(\w)\1{2,}', '', word) if re.search(r'(\w)\1{2,}', word) else word for word in words]

        # Convert to lowercase and remove punctuations.
        words = [word.lower().strip() for word in words]

        # Substitute the tokens with "" where they are just numbers.
        words = ['' if word.isdigit() else word for word in words]

        # Join the words back into a single string.
        text = ' '.join([word for word in words if word])
    
    return text

In [7]:
%%time
cfpb['clean_narr'] = cfpb['Consumer complaint narrative'].apply(text_normalizer)
cfpb

Wall time: 10.1 s


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,clean_narr
4,05/31/23,Debt collection,Auto debt,Written notification about debt,Didn't receive enough information to verify debt,"U.S.C in Violations 1. Title 15 USC 689n ( a ) ( b ) Unlawful acts and Omissions, breach of fiduciary duty 2. Title 12 Ch.X Part 1006.34 ( a ) ( c ), c ( 2 ) ( v ), c ( 3 ) ( ii ) Validation Information Required 3. Title 18 USC 1001 a ( 1 ) Statements or entries in generally 4. Title 15 USC 1681s-2 ( a ) ( b ), ( f ) ( 2 ) ( b ) ( f ) ( 3 ) Responsibilities of Furnishers of information to consumer reporting agencies 5. Title 15 USC 1611 ( 1 ) Criminal liability for willful and knowing violation 6. Title 18 USC 242 Deprivation of rights under color of law Denied Rights to Discovery 7.Title 15 USC 1691 ( a ), ( a ) ( 3 ) Scope of Prohibition 8. Title 15 USC 1681o, ( a ), ( a ) ( 1 ) Civil Liability for negligent noncompliance Requested verification Information. \n1. They did not produce a promissory note ; there was no verification. Noncompliance 2. Did they produce a contract signed by XXXX XXXX XXXX and by XXXX ; No verification or disclosure was provided. Noncompliance 3. Did they produce any verification of their claim ; affidavit/signed invoice was not provided willfully. \n\n4. When asked who the original creditor was, did they state clearly whom it was and their address ; they willfully denied full disclosure and failed to give proper verification information. Noncompliance 5. There was no Unreserved acceptance by both parties. ( I reserved my rights to rescind ) 6. Was equal consideration exchanged? No disclosure or verification was given. \n\n\n7. Have any other accounts been opened in my name at my address with or without my consent? No disclosure or verification was given. \n\n8. They disclosed my personal information to consumer reporting agencies. They reported under three different names : one name per agency. Reporting unverified incorrect information causing damage. \n\n\n9. Has the account been securitized? What is the Cusip-number? No disclosure or verification given. Noncompliance",Company has responded to the consumer and the CFPB and chooses not to provide a public response,Bridgecrest Acceptance Corporation,AL,361XX,None,Consent provided,Web,05/31/23,Closed with explanation,Yes,NaN,7045946,u s c in violations title usc 689n a b unlawful acts and omissions breach of fiduciary duty title ch x part a c c v c ii validation information required title usc a statements or entries in generally title usc 1681s-2 a b f b f responsibilities of furnishers of information to consumer reporting agencies title usc criminal liability for willful and knowing violation title usc deprivation of rights under color of law denied rights to discovery title usc a a scope of prohibition title usc 1681o a a civil liability for negligent noncompliance requested verification information they did not produce a promissory note there was no verification noncompliance did they produce a contract signed by and by no verification or disclosure was provided noncompliance did they produce any verification of their claim affidavit signed invoice was not provided willfully when asked who the original creditor was did they state clearly whom it was and their address they willfully denied full disclosure and failed to give proper verification information noncompliance there was no unreserved acceptance by both parties i reserved my rights to rescind was equal consideration exchanged no disclosure or verification was given have any other accounts been opened in my name at my address with or without my consent no disclosure or verification was given they disclosed my personal information to consumer reporting agencies they reported under three different n

In [8]:
cfpb['clean_narr'].sample(5)

213498                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               in i purchased a ce

In [9]:
def vectorize_text(text):
    vectors = [embeddings_dict_6B_50D.get(word) for word in str(text).split() if word in embeddings_dict_6B_50D]
    vectors = [v for v in vectors if v is not None]  # remove any None values
    if vectors:
        vectorized = np.mean(vectors, axis=0)
    else:
        vectorized = np.zeros(50)  # if there are no vectors, return a zero-vector
    return vectorized

In [10]:
vectorized_texts = cfpb['clean_narr'].apply(vectorize_text)

# convert vectorized_texts into a DataFrame
vector_df = pd.DataFrame(vectorized_texts.tolist(), columns=[f'glove_{i}' for i in range(1, 51)])

# reset the index of both DataFrames to ensure proper concatenation
cfpb = cfpb.reset_index(drop=True)
vector_df = vector_df.reset_index(drop=True)

# concatenate the original DataFrame cfpb and vector_df
cfpb = pd.concat([cfpb, vector_df], axis=1)

In [11]:
cfpb.sample(5)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,glove_41,glove_42,glove_43,glove_44,glove_45,glove_46,glove_47,glove_48,glove_49,glove_50
27330,05/05/23,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Problem with a credit reporting company's investigation into an existing problem,Their investigation did not fix an error on your report,"When I reviewed my credit report, I discovered that some of the information was erroneous. The TransUnion credit bureau must validate these items in line with Sections 609 ( a ) ( 1 ) ( A ) and 611 ( a ) ( 1 ) ( A ). It is not authorized to fail to validate this reporting items as unverified information without providing any proof within the period allowed by law. The following item need to investigate and correct for erroneous reporting in my file and are listed on my credit report : XXXX XXXX XXXX XXXX XXXX",Company has responded to the consumer and the CFPB and chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,90247,...,-0.002492,-0.003829,0.166547,0.223561,-0.193782,-0.032961,-0.108569,0.347119,0.169034,-0.061433
33325,04/18/23,Debt collection,Medical debt,Attempts to collect debt not owed,Debt was result of identity theft,"I checked my credit report and seen that there were discrepancies and there is a charge of {$170.00} and it was started on XX/XX/2020, the account number is XXXX. I've contacted the company, and nothing was done yet this money keeps going up in interest and this is still on my credit which is hurting my credit and good name.",Company believes it acted appropriately as authorized by contract or law,Creditors Bureau Associates,GA,30813,...,-0.138108,0.039409,0.023946,0.089534,-0.130182,-0.080635,-0.161845,0.005993,0.063838,0.013127
49721,05/01/23,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Problem with a credit reporting company's investigation into an existing problem,Their investigation did not fix an error on your report,"Trans Union is reporting an inaccurate and inconsistent information in my credit report. I have disputed the inaccuracy on many occasions but Trans Union simply send me a letter that states account verified as accurate without providing me with the description of the procedure used to determine the accuracy and completeness of the information contained in my credit report as required by the FCRA. \nI have requested the description of the procedure used to determine the accuracy of this account from Trans Union on XX/XX/XXXX, XXXX, XXXX, XX/XX/XXXX and XX/XX/XXXX but have not received anything back at the time of filing this complaint. \n\nTrans Union is therefore in violation of 15 U.S. Code 1681i ( a ) ( 6 ) ( B ) ( iii ) which states : a notice that, if requested by the consumer, a description of the procedure used to determine the accuracy and completeness of the information shall be provided to the consumer by the agency, including the business name and address of any furnisher of information contacted in connection with such information and the telephone number of such furnisher, if reasonably available",Company has responded to the consumer and the CFPB and chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",ND,XXXXX,...,0.030034,-0.007168,0.128215,0.144809,-0.223915,0.041257,-0.114796,0.281004,0.037676,-0.075147
50654,04/13/23,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Problem with a credit reporting company's investigation into an existing problem,Their investigation did not fix an error on your report,"The following accounts continue to report on my credit report without my permission, in violation of federal law, despite numerous written requests. As required by Sections 609 ( a ) ( 1 ) ( A ) and 611 ( a ) ( 1 ), these accounts must be verified ( 

In [12]:
cfpb.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    37872
Debt collection                                                                  3971
Credit card or prepaid card                                                      3470
Checking or savings account                                                      3382
Mortgage                                                                         1633
Vehicle loan or lease                                                             972
Money transfer, virtual currency, or money service                                818
Payday loan, title loan, or personal loan                                         403
Student loan                                                                      245
Name: Product, dtype: int64

In [13]:
cfpb['is_debt_collection'] = cfpb.Product.apply(lambda x: 1 if x=='Debt collection' else 0)

In [14]:
from sklearn.model_selection import train_test_split

# Assuming 'is_debt_collection' is the column that specifies if a complaint is about debt collection
y = cfpb['is_debt_collection']
X = cfpb.loc[:, 'glove_1':'glove_50']  # Extract only the GloVe vector columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from imblearn.over_sampling import SMOTE, RandomOverSampler

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
# oversample = RandomOverSampler(sampling_strategy='minority')
# X_train_smote, y_train_smote = oversample.fit_resample(X_train, y_train)

In [16]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(class_weight='balanced')

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}

In [17]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(logreg, param_grid, cv=5, scoring='f1')

grid.fit(X_train_smote, y_train_smote)

C:\Users\JYM\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\JYM\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

C:\Users\JYM\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\JYM\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

GridSearchCV(cv=5, estimator=LogisticRegression(class_weight='balanced'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [18]:
from sklearn.metrics import classification_report

y_pred = grid.predict(X_test)

print(classification_report(y_test, y_pred))

# Print the mean test scores and parameters for each iteration
results = grid.cv_results_
for mean_test_score, params in zip(results["mean_test_score"], results["params"]):
    print(params, "has a score of", mean_test_score)

              precision    recall  f1-score   support

           0       0.98      0.79      0.88      9747
           1       0.24      0.78      0.37       807

    accuracy                           0.79     10554
   macro avg       0.61      0.79      0.62     10554
weighted avg       0.92      0.79      0.84     10554

{'C': 0.001, 'penalty': 'l1'} has a score of nan
{'C': 0.001, 'penalty': 'l2'} has a score of 0.7154709816044507
{'C': 0.01, 'penalty': 'l1'} has a score of nan
{'C': 0.01, 'penalty': 'l2'} has a score of 0.7790455296170328
{'C': 0.1, 'penalty': 'l1'} has a score of nan
{'C': 0.1, 'penalty': 'l2'} has a score of 0.8054920795218459
{'C': 1, 'penalty': 'l1'} has a score of nan
{'C': 1, 'penalty': 'l2'} has a score of 0.8111470687284289
{'C': 10, 'penalty': 'l1'} has a score of nan
{'C': 10, 'penalty': 'l2'} has a score of 0.8134481487933186
{'C': 100, 'penalty': 'l1'} has a score of nan
{'C': 100, 'penalty': 'l2'} has a score of 0.81368117698367
{'C': 1000, 'penalty'

In [28]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define the model with class weights
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Define the hyperparameters
# param_grid = {
#     'n_estimators': [50, 70, 100],
#     'max_features': ['sqrt', 'log2'],
#     'max_depth' : [4,6,8],
#     'criterion' :['gini', 'entropy']
# }
# # best is {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 70} has a score of 0.36957725010492326

param_grid = {
    'n_estimators': [70, 100, 125],
    'max_features': ['sqrt'],
    'max_depth' : [8,10,12],
    'criterion' :['gini']
}

# Perform hyperparameter tuning
grid = GridSearchCV(rf, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

# Evaluate your model
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

# Perform hyperparameter tuning
grid = GridSearchCV(rf, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

# Print the mean test scores and parameters for each iteration
results = grid.cv_results_
for mean_test_score, params in zip(results["mean_test_score"], results["params"]):
    print(params, "has a score of", mean_test_score)


              precision    recall  f1-score   support

           0       0.96      0.95      0.95      9747
           1       0.44      0.49      0.46       807

    accuracy                           0.91     10554
   macro avg       0.70      0.72      0.71     10554
weighted avg       0.92      0.91      0.92     10554

{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 70} has a score of 0.36957725010492326
{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 100} has a score of 0.36934211332761724
{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 125} has a score of 0.36962229571741123
{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 70} has a score of 0.43226297519003376
{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100} has a score of 0.4382638819859258
{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 125} 

In [31]:
%%time
from sklearn.svm import SVC
from sklearn.utils import class_weight

# Define the model with class weights
svm = SVC(class_weight='balanced')

# # Define the hyperparameters
# param_grid = {'C': [0.01, 0.1, 1, 10], 
#               'gamma': [1, 0.01, 0.0001],
#               'kernel': ['linear', 'poly', 'rbf']}
# # best is about 0.5

# param_grid = {'C': [10,50,100], 
#               'gamma': [1, 0.8, 0.9],
#               'kernel': ['rbf']}
# # {'C': 100, 'gamma': 1, 'kernel': 'rbf'} has a score of 0.5484671199176903

# param_grid = {'C': [100,500,1000], 
#               'gamma': [1],
#               'kernel': ['rbf']}
# # {'C': 500, 'gamma': 1, 'kernel': 'rbf'} has a score of 0.5564294078533358

param_grid = {'C': [250,500,750], 
              'gamma': [0.5, 1],
              'kernel': ['rbf']}

# Perform hyperparameter tuning
grid = GridSearchCV(svm, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

# Evaluate your model
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

# Print the mean test scores and parameters for each iteration
results = grid.cv_results_
for mean_test_score, params in zip(results["mean_test_score"], results["params"]):
    print(params, "has a score of", mean_test_score)

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      9747
           1       0.52      0.67      0.58       807

    accuracy                           0.93     10554
   macro avg       0.74      0.81      0.77     10554
weighted avg       0.94      0.93      0.93     10554

{'C': 250, 'gamma': 0.5, 'kernel': 'rbf'} has a score of 0.5208619565580647
{'C': 250, 'gamma': 1, 'kernel': 'rbf'} has a score of 0.557634708363124
{'C': 500, 'gamma': 0.5, 'kernel': 'rbf'} has a score of 0.5328856868991231
{'C': 500, 'gamma': 1, 'kernel': 'rbf'} has a score of 0.5564294078533358
{'C': 750, 'gamma': 0.5, 'kernel': 'rbf'} has a score of 0.5410453873309635
{'C': 750, 'gamma': 1, 'kernel': 'rbf'} has a score of 0.5495397852588717
Wall time: 1h 27min 9s


In [25]:
!pip install xgboost

     --------------------------------------- 70.9/70.9 MB 25.2 MB/s eta 0:00:00


In [30]:
%%time

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define the model
xgb = XGBClassifier()

# Define the hyperparameters
# param_grid = {
#     'n_estimators': [50, 70, 100],
#     'max_depth' : [4,6,8],
#     'learning_rate' : [0.01, 0.05, 0.1]
# }
# # {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100} has a score of 0.40803526169579774

# https://amangupta16.medium.com/xgboost-hyperparameters-explained-bb6ce580501d
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [6, 8, 10],
    'grow_policy':['depthwise', 'lossguide'],
    'learning_rate': [0.01, 0.1, 0.25, 0,5],
    'gamma': [0, 0.25, 0.5, 0.75, 1.0],
    'objective': ['binary:logistic'],
#     'booster':['gbtree', 'gblinear', 'dart'], 
#     'min_child_weight': [0, 10, 100],
#     'max_delta_step': [0, 5, 10],
#     'subsample': [0.5, 0.75, 1.0],
#     'colsample_bytree': [0.4, 0.6, 0.8, 1.0],
#     'reg_alpha': [0, 0.1, 0.5, 1.0],
#     'reg_lambda': [0, 0.1, 0.5, 1.0],
#     'scale_pos_weight': [0.1, 0.25, 1]  
}
# {'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4600636603662126
# {'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.47833576908629316
# {'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.481646301347

# {'gamma': 0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4600636603662126
# {'gamma': 0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.47833576908629316
# {'gamma': 0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.481646301347

# {'gamma': 0.25, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4502555747462524
# {'gamma': 0.25, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4723017880098485
# {'gamma': 0.25, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.47917677346322646

# {'gamma': 0.5, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.45447253060264303
# {'gamma': 0.5, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4731291506961715
# {'gamma': 0.5, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.4777666298




# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4519261276161095
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4717137003276105
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.4717137003276105
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4569095910751261
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.45770395065209435
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.45770395065209435
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# {'gamma': 1.0, 'grow_policy': 'depthwise', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4519261276161095
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4717137003276105
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.4717137003276105
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4569095910751261
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.45770395065209435
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.45770395065209435
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# {'gamma': 1.0, 'grow_policy': 'lossguide', 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.4445592148570988
# Perform hyperparameter tuning
grid = GridSearchCV(xgb, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

# Evaluate your model
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

# Print the mean test scores and parameters for each iteration
results = grid.cv_results_
for mean_test_score, params in zip(results["mean_test_score"], results["params"]):
    print(params, "has a score of", mean_test_score)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      9747
           1       0.76      0.38      0.51       807

    accuracy                           0.94     10554
   macro avg       0.86      0.68      0.74     10554
weighted avg       0.94      0.94      0.93     10554

{'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.10238196592887869
{'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200, 'objective': 'binary:logistic'} has a score of 0.12326588238927161
{'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 300, 'objective': 'binary:logistic'} has a score of 0.16790070484041877
{'gamma': 0, 'grow_policy': 'depthwise', 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 100, 'objective': 'binary:logistic'} has a score of 0.19072548145183887
{

In [22]:
%%time
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define the model
gnb = GaussianNB()

# Define the hyperparameters
# Note: these values are just for example and should be adjusted for your specific problem
param_grid = {'var_smoothing': np.logspace(0,-9, num=20)}
# {'var_smoothing': 0.004281332398719396} has a score of 0.24493749460487274

# Perform hyperparameter tuning
grid = GridSearchCV(gnb, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

# Evaluate your model
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))


# Print the mean test scores and parameters for each iteration
results = grid.cv_results_
for mean_test_score, params in zip(results["mean_test_score"], results["params"]):
    print(params, "has a score of", mean_test_score)

              precision    recall  f1-score   support

           0       0.96      0.73      0.83      9747
           1       0.16      0.61      0.25       807

    accuracy                           0.72     10554
   macro avg       0.56      0.67      0.54     10554
weighted avg       0.90      0.72      0.78     10554

{'var_smoothing': 1.0} has a score of 0.031751898762184386
{'var_smoothing': 0.3359818286283782} has a score of 0.22271121433197316
{'var_smoothing': 0.11288378916846892} has a score of 0.23784452576747675
{'var_smoothing': 0.0379269019073225} has a score of 0.24196490807455256
{'var_smoothing': 0.012742749857031341} has a score of 0.24448418950709527
{'var_smoothing': 0.004281332398719396} has a score of 0.24493749460487274
{'var_smoothing': 0.0014384498882876629} has a score of 0.24497064328551715
{'var_smoothing': 0.0004832930238571757} has a score of 0.24489646871311038
{'var_smoothing': 0.00016237767391887227} has a score of 0.24478835362982004
{'var_smoothing

In [23]:
# # !pip install --upgrade gensim
# # !pip install gensim==3.4.0
# # !pip install smart_open==1.9.0

# # we are not gonno use gensim here due to library version conflict

# # Load the GloVe model.
# glove_input_file = "glove.6B/glove.6B.50d.txt"
# word2vec_output_file = "glove.6B/gensim_glove_vectors.6B.50d.txt"
# glove2word2vec(glove_input_file, word2vec_output_file)
# glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

# # Assuming your DataFrame is named df and your text column is named 'complaint'
# df = pd.DataFrame({
#     'complaint': ['This is the first complaint.', 'And this is the second one.']
# })

# # Preprocess text
# stop_words = set(stopwords.words('english'))
# df['complaint'] = df['complaint'].str.lower().apply(lambda x: ''.join(ch for ch in x if ch not in string.punctuation))
# df['complaint'] = df['complaint'].apply(lambda x: word_tokenize(x))
# df['complaint'] = df['complaint'].apply(lambda x: [word for word in x if word not in stop_words])

# # Replace each word in your 'complaint' column with its corresponding vector.
# df['complaint_vector'] = df['complaint'].apply(lambda x: [glove_model[word] if word in glove_model else np.zeros((50,)) for word in x])
